In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
from datetime import datetime
import requests
import pandas_datareader as pdr
from datetime import datetime
import pandas_datareader as pdr
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
%matplotlib inline
warnings.filterwarnings('ignore')

/Users/gregoryrash/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Function to Validate Tickers
def validateTick(tick):
    
    # Try Inputting into Yahoo Stocks
    try:
        pdr.DataReader(f'{tick}','yahoo',date)
    
    # Raise Error if Invalid
    except:
        raise ValueError("Ticker Not Valid.")
        
# Function to Validate Date
def validate(date_text):
    
    # Try Check on Format
    try:
        datetime.strptime(date_text, '%Y-%m-%d')
        
    # Raise Error if Invalid
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")
        
# Function to Validate Date
def validateShift(shift):
    shift = int(shift)
    # Try Check on Format
    try:
        if 0 > shift <= 365:
            pass
        
    # Raise Error if Invalid
    except ValueError:
        raise ValueError("Please enter a whole number.")

In [3]:
# Pick a Start Date
date = input("Pick a start date for your analysis (YYYY-MM-DD): ")

# Validate Correct Date Format
validate(date)

Pick a start date for your analysis (YYYY-MM-DD):  1980-01-01


In [4]:
# Pick a Stock
stock_selection = input("Pick a stock ticker for your analysis: ")

# Validate Correct Date Format
validateTick(stock_selection)

Pick a stock ticker for your analysis:  MSFT


In [5]:
# Pick Shift
shift_selection = input("Pick how many days to shift your close values for your analysis: ")

# Validate Correct Date Format
validateShift(shift_selection)

Pick how many days to shift your close values for your analysis:  1


In [6]:
# Function to Create Stocks
def stockCreation(date, stock_selection, days_shifted):
    
    # Read Stock Data
    globals()[stock_selection] = pdr.DataReader(f'{stock_selection}','yahoo',date)
    
    # Create Shifted Close Column
    globals()[stock_selection]["Shifted Close"] = globals()[stock_selection]["Adj Close"].shift(-int(days_shifted))
    
    # Drop Rows Without Shift Values
    globals()[stock_selection].drop(globals()[stock_selection].tail(int(days_shifted)).index, inplace = True) 
    
    # Empty List to Hold Profit or Loss
    profit_loss = []
    
    # Iterate Through DataFrame
    for x in range(len(globals()[stock_selection])):
        
        # Handle Profitable Days
        if globals()[stock_selection]["Shifted Close"][x] > globals()[stock_selection]["Adj Close"][x]:
            
            # Append "Profit" to Empty List
            profit_loss.append("Profit")
            
        # Handle Loss Days
        elif globals()[stock_selection]["Shifted Close"][x] < globals()[stock_selection]["Adj Close"][x]:
            
            # Append "Loss" to Empty List
            profit_loss.append("Loss")
            
        # Handle Push Days
        elif globals()[stock_selection]["Shifted Close"][x] == globals()[stock_selection]["Adj Close"][x]:
            
            # Append "Push" to Empty List
            profit_loss.append("Push")
    
    # Create Profit/Loss Column
    globals()[stock_selection]["Profit/Loss"] = profit_loss
    
    # Create Returns Column
    globals()[stock_selection]['Returns'] = np.log(globals()[stock_selection]['Adj Close'] / globals()[stock_selection]['Adj Close'].shift(1))
    
    # Iterate to Create STD Columns
    for x in range(5, 20, 5):
        
        # Create STD Column Based on Iteration
        col_name = 'std_' + str(x)
        
        # Add STD Column to DataFrame
        globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).std()

    # Iterate to Create MA Columns
    for x in range(10, 30, 5):
        
        # Create MA Column Based on Iteration
        col_name = 'ma_' + str(x)
        
        # Add MA Column to DataFrame
        globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).mean()

    # Iterate to Create % Change Columns
    for x in range(3, 12, 3):
        
        # Create % Change Column Based on Iteration
        col_name = 'pct_' + str(x)
        
        # Add % Change Column to DataFrame
        globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].pct_change().rolling(window=x).sum()

    # Create VMA 4 Column
    col_name = 'vma_4'
    
    # Add VMA 4 Column to DataFrame
    globals()[stock_selection][col_name] = globals()[stock_selection]['Volume'].rolling(4).mean()

    # Create Intraday Column
    col_name = 'co'
    
    # Add Intraday Column to DataFrame
    globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'] - globals()[stock_selection]['Open']
    
    # Drop All NaN's
    globals()[stock_selection].dropna(inplace=True)
    
    # Return DataFrame
    return globals()[stock_selection]

In [7]:
# ### Function to Create Stocks
# def stockCreation(date, stock_selection, days_shifted):
    
#     # Read Stock Data
#     globals()[stock_selection] = pdr.DataReader(f'{stock_selection}','yahoo',date)
    
#     # Create Shifted Close Column
#     globals()[stock_selection]["Shifted Close"] = globals()[stock_selection]["Adj Close"].shift(-int(days_shifted))
    
#     # Drop Rows Without Shift Values
#     globals()[stock_selection].drop(globals()[stock_selection].tail(int(days_shifted)).index, inplace = True) 
    
#     # Empty List to Hold Profit or Loss
#     profit_loss = []
    
#     # Iterate Through DataFrame
#     for x in range(len(globals()[stock_selection])):
        
#         # Save Adjusted Close Value
#         adj_close = globals()[stock_selection]["Adj Close"][x]
        
#         # Save Shifted Close Value
#         shifted_close = globals()[stock_selection]["Shifted Close"][x]
        
#         # Handle Returns 3x as Large as Open
#         if shifted_close > (3 * adj_close):
            
#             # Append Number 8
#             profit_loss.append(8)
            
#         # Handle Returns within 2x and 3x as Large as Open
#         elif (3 * adj_close) <= shifted_close > (2 * adj_close):
            
#             # Append Number 7
#             profit_loss.append(7)
            
#         # Handle Returns within 1.75x and 2x as Large as Open
#         elif (2 * adj_close) <= shifted_close > (1.75 * adj_close):
            
#             # Append Number 6 
#             profit_loss.append(6)
        
#         # Handle Returns within 1.5x and 1.75x as Large as Open
#         elif (1.75 * adj_close) <= shifted_close > (1.5 * adj_close):
            
#             # Append Number 5
#             profit_loss.append(5)
            
#         # Handle Returns within 1,25x and 1.5x as Large as Open
#         elif (1.5 * adj_close) <= shifted_close > (1.25 * adj_close):
            
#             # Append Number 4
#             profit_loss.append(4)
            
#         # Handle Returns within 1.1x and 1.25x as Large as Open
#         elif (1.25 * adj_close) <= shifted_close > (1.1 * adj_close):
            
#             # Append Number 3
#             profit_loss.append(3)
            
#         # Handle Returns within 1.05x and 1.1x as Large as Open
#         elif (1.1 * adj_close) <= shifted_close > (1.05 * adj_close):
            
#             # Append Number 2
#             profit_loss.append(2)
            
#         # Handle Returns within 1x and 1.05x as Large as Open
#         elif (1.05 * adj_close) <= shifted_close > (1 * adj_close):
            
#             # Append Number 1
#             profit_loss.append(1)
            
#         # Handle Returns within .95x and 1x as Small as Open
#         elif (1 * adj_close) <= shifted_close > (.95 * adj_close):
            
#             # Append Number 0
#             profit_loss.append(0)
        
#         # Handle Returns within .9x and .95x as Small as Open
#         elif (.95 * adj_close) <= shifted_close > (.9 * adj_close):
            
#             # Append Number -1
#             profit_loss.append(-1)
            
#         # Handle Returns within .75x and .9x as Small as Open
#         elif (.9 * adj_close) <= shifted_close > (.75 * adj_close):
            
#             # Append Number -2
#             profit_loss.append(-2)
            
#         # Handle Returns within .5x and .75x as Small as Open
#         elif (.75 * adj_close) <= shifted_close > (.5 * adj_close):
            
#             # Append Number -3
#             profit_loss.append(-3)
            
#         # Handle Returns within .25x and .5x as Small as Open
#         elif (.5 * adj_close) <= shifted_close > (.25 * adj_close):
            
#             # Append Number -4
#             profit_loss.append(-4)
            
#         # Handle Returns within .1x and .25x as Small as Open
#         elif (.25 * adj_close) <= shifted_close > (.1 * adj_close):
            
#             # Append Number -5
#             profit_loss.append(-5)
            
#         # Handle Returns within .05x and .1x as Small as Open
#         elif (.1 * adj_close) <= shifted_close > (.05 * adj_close):
            
#             # Append Number -6
#             profit_loss.append(-6)
        
#         # Handle Returns Smaller than .05x as Small as Open
#         elif shifted_close < (.05 * adj_close):
            
#             # Append Number -7
#             profit_loss.append(-7)
    
#     # Create Profit/Loss Column
#     globals()[stock_selection]["Profit/Loss"] = profit_loss
    
#     # Create Returns Column
#     globals()[stock_selection]['Returns'] = np.log(globals()[stock_selection]['Adj Close'] / globals()[stock_selection]['Adj Close'].shift(1))
    
#     # Iterate to Create STD Columns
#     for x in range(5, 20, 5):
        
#         # Create STD Column Based on Iteration
#         col_name = 'std_' + str(x)
        
#         # Add STD Column to DataFrame
#         globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).std()

#     # Iterate to Create MA Columns
#     for x in range(10, 30, 5):
        
#         # Create MA Column Based on Iteration
#         col_name = 'ma_' + str(x)
        
#         # Add MA Column to DataFrame
#         globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).mean()

#     # Iterate to Create % Change Columns
#     for x in range(3, 12, 3):
        
#         # Create % Change Column Based on Iteration
#         col_name = 'pct_' + str(x)
        
#         # Add % Change Column to DataFrame
#         globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].pct_change().rolling(window=x).sum()

#     # Create VMA 4 Column
#     col_name = 'vma_4'
    
#     # Add VMA 4 Column to DataFrame
#     globals()[stock_selection][col_name] = globals()[stock_selection]['Volume'].rolling(4).mean()

#     # Create Intraday Column
#     col_name = 'co'
    
#     # Add Intraday Column to DataFrame
#     globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'] - globals()[stock_selection]['Open']
    
#     # Drop All NaN's
#     globals()[stock_selection].dropna(inplace=True)
    
#     # Return DataFrame
#     return globals()[stock_selection]

In [8]:
# Create Stock
stockCreation(date, stock_selection, shift_selection)

,High,Low,Open,Close,Volume,Adj Close,Shifted Close,Profit/Loss,Returns,std_5,...,std_15,ma_10,ma_15,ma_20,ma_25,pct_3,pct_6,pct_9,vma_4,co
Date,,,,,,,,,,,,,,,,,,,,,
1986-04-17,0.105035,0.104167,0.104167,0.105035,22003200.0,0.066867,0.064656,Loss,0.008298,0.001493,...,0.002125,0.063164,0.062409,0.061617,0.061981,0.042823,0.078144,0.087482,18842400.0,-0.037300
1986-04-18,0.105035,0.100694,0.105035,0.101563,21628800.0,0.064656,0.064656,Push,-0.033614,0.001296,...,0.002177,0.063496,0.062630,0.061810,0.062092,0.009768,0.036161,0.072444,21211200.0,-0.040379
1986-04-21,0.102431,0.098958,0.101563,0.101563,22924800.0,0.064656,0.063551,Loss,0.000000,0.001198,...,0.002173,0.063938,0.062888,0.062086,0.062114,-0.024723,0.018463,0.063270,24616800.0,-0.036907
1986-04-22,0.101563,0.099826,0.101563,0.099826,15552000.0,0.063551,0.063827,Profit,-0.017251,0.001354,...,0.002049,0.064214,0.063109,0.062390,0.062048,-0.050158,-0.007335,0.027986,20527200.0,-0.038012
1986-04-23,0.100694,0.098958,0.099826,0.100260,15609600.0,0.063827,0.070182,Profit,0.004338,0.001302,...,0.001951,0.064407,0.063311,0.062653,0.062059,-0.012755,-0.002988,0.023406,18928800.0,-0.035999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-03,245.089996,239.259995,239.570007,243.000000,27158100.0,243.000000,242.009995,Loss,0.014466,4.046248,...,10.105464,233.872998,228.101332,225.204998,224.317598,0.047139,0.046312,0.078711,32214375.0,3.429993
2021-02-04,243.240005,240.369995,242.660004,242.009995,25296100.0,242.009995,242.199997,Profit,-0.004082,4.331535,...,10.144941,235.576997,229.812665,226.692998,225.031998,0.009913,0.039785,0.070281,27921175.0,-0.650009
2021-02-05,243.279999,240.419998,242.229996,242.199997,18043900.0,242.199997,242.470001,Profit,0.000785,1.591175,...,9.470124,237.201997,231.757998,227.888498,225.852798,0.011282,0.014679,0.055222,24103600.0,-0.029999


In [9]:
# Assign Y Values
y = stockCreation(date, stock_selection, shift_selection)["Profit/Loss"]

In [10]:
# Assign X Values
X = stockCreation(date, stock_selection, shift_selection).drop(["Shifted Close", "Profit/Loss"], axis=1)

In [11]:
# Train/Test Split 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)